In [ ]:
from dotenv import load_dotenv
import sys
import os

from typing import Annotated, Literal, TypedDict, Sequence, Optional, List
from pydantic import BaseModel, Field

from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage, AnyMessage


from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

sys.path.append(os.path.abspath(".."))
from utils.memory_store import MemoryStore

In [2]:
load_dotenv()

True

In [3]:
store = MemoryStore()
USER_ID = "3"
key = "semantic_memory"

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    personal_info_detected: Literal["yes", "no"]
    delete_request: Literal["yes", "no"]
    personal_info_extracted: Optional[List[str]] # this stores entity for either to delete or add
    new_info: Optional[str]
    collected_memories: Optional[str]

In [5]:
class ClassifyInformation(BaseModel):
    personal_info: Literal["yes", "no"] = Field(
        description="Indicates whether the information contains any personal user choices or preferences that could help an LLM provide more personalized responses over time for long-term memory use."
    )

In [6]:
def personal_info_classifier(state: AgentState) -> AgentState:
    """
    Classifies if the last user message contains personal info.
    Now also detects if user is forgetting or updating past preferences.
    """
    message = state["messages"][-1].content

    system_prompt = """You are a classifier that checks if a message contains personal info.

Personal info includes:
- Names (e.g., "John Smith")
- Locations (e.g., "Berlin", "123 Main St")
- Preferences or hobbies (e.g., "I love to code", "I prefer short replies")
- Tools and Technologies Used  (e.g., "I mostly use PyTorch and FastAPI", "I am using Windows 11 now")
- Dislikes or updates to previous preferences (e.g., "I do not use Magnet anymore", "I stopped liking Twitter")
- Occupation

Respond "yes" if the message reveals or updates any personal preferences, locations, names, etc.
Respond "no" if it is a general query or lacks any personal information.

Examples:
User: "My name is Thomas, I live in Vancouver."
Classifier: "yes"

User: "I love pizza with extra cheese."
Classifier: "yes"

User: "I no longer use Discord."
Classifier: "yes"

User: "I do not like tea anymore."
Classifier: "yes"

User: "I still use Notion daily."
Classifier: "yes"

User: "What is the capital of France?"
Classifier: "no"

User: "This is great weather."
Classifier: "no"

User: "Hello, how are you?"
Classifier: "no"
"""

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", "{message}"),
    ])

    llm = ChatOpenAI(model="gpt-4o-mini", max_completion_tokens=50)
    structured_llm = llm.with_structured_output(ClassifyInformation)
    chain = prompt | structured_llm

    result = chain.invoke({"message": message})
    state["personal_info_detected"] = result.personal_info

    return state


In [7]:
def personal_info_router(state: AgentState) -> Literal["classify_add_or_delete", "retrieve_memories"]:
    """
    If personal info is detected, route to the node that classifies whether to add or delete it.
    Otherwise, route to retrieving memories.
    """
    if state["personal_info_detected"].lower() == "yes":
        return "classify_add_or_delete"
    return "retrieve_memories"

In [8]:
class DeleteRequest(BaseModel):
    delete_request: Literal["yes", "no"] = Field(
        description="Return 'yes' if the user wants to delete or stop using something previously shared, otherwise 'no'."
    )

def classify_add_or_delete(state: AgentState) -> AgentState:
    """
    Classifies whether the personal information is a delete request or an addition.
    """
    last_message = state["messages"][-1].content
    # print(f"last message: {last_message}")

    system_prompt = """You are a classifier that decides whether a user's message indicates a request to delete or stop using something.

    Return "yes" if the user says they:
    - No longer use something
    - Have stopped liking a tool, app, or preference
    - Want to remove or undo a previously stated preference

    Return "no" if the user is sharing a new preference, habit, tool, or hobby.

    Examples:
    User: "I do not use Facebook anymore."
    Classifier: "yes"

    User: "I stopped liking pineapple on pizza."
    Classifier: "yes"

    User: "I enjoy hiking on weekends."
    Classifier: "no"

    User: "I love using Notion for productivity."
    Classifier: "no"
    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_prompt),
        ("human", last_message),
    ])

    llm = ChatOpenAI(model="gpt-4o-mini", max_completion_tokens=50)
    structured_llm = llm.with_structured_output(DeleteRequest)
    chain = prompt | structured_llm

    result = chain.invoke({"message": prompt})
    # print(f"Result: {result}")
    state["delete_request"] = result.delete_request
    return state


In [9]:
classify_add_or_delete({"messages": [HumanMessage(content="hi, I don't want to use magnet anymore")]})

{'messages': [HumanMessage(content="hi, I don't want to use magnet anymore", additional_kwargs={}, response_metadata={})],
 'delete_request': 'yes'}

In [10]:
def route_add_or_delete(state: AgentState) -> Literal["extract_delete_entity", "personal_info_extractor"]:
    """
    If user intends to delete info, route to extract_delete_entity.
    Otherwise, route to personal_info_extractor for new additions.
    """
    if state["delete_request"].lower() == "yes":
        return "extract_delete_entity"
    return "personal_info_extractor"

In [11]:
system_prompt = """You are an assistant that extracts the names of tools, preferences, or technologies the user wants to forget or delete from memory. Only return the list of entities to remove."""

few_shot_examples = [
    {
        "input": "I no longer use Twitter.",
        "output": '["Twitter"]'
    },
    {
        "input": "Forget Notion and Todoist.",
        "output": '["Notion", "Todoist"]'
    },
    {
        "input": "I stopped liking pizza and burgers.",
        "output": '["pizza", "burgers"]'
    },
    {
        "input": "I do not use Figma, Zoom, or Slack anymore.",
        "output": '["Figma", "Zoom", "Slack"]'
    },
]
    
def format_prompt(user_message: str) -> List:
    few_shots = "\n".join(
        [f"Input: {ex['input']}\nOutput: {ex['output']}" for ex in few_shot_examples]
    )
    return [
        SystemMessage(content=system_prompt + "\n\n" + few_shots),
        HumanMessage(content=f"Input: {user_message}\nOutput:")
    ]

class EntitiesToForget(BaseModel):
    entities: List[str] = Field(..., description="List of entities the user wants to delete or forget")

def extract_delete_entity(state: AgentState) -> AgentState:
    messages = state["messages"][-1].content
    user_input = messages # Get last user message content

    # System instructions with few-shot examples
    instruction = """
    You are an intelligent assistant that helps identify which pieces of personal information the user wants to delete from memory.

    Instructions:
    - Analyze the input and extract the names of entities, preferences, or facts the user wants the system to forget.
    - Return only a list of strings, each representing one such item.

    Examples:
    User: "Forget that I live in Delhi and that I work at Microsoft."
    Output: ["Location: Delhi", "Employer: Microsoft"]

    User: "Remove everything about my cat and my love for sushi."
    Output: ["Pet: cat", "Food Preference: sushi"]

    User: "Never store my email or my travel plans to Japan."
    Output: ["Email", "Travel Plan: Japan"]

    User: "Just chatting, nothing to delete."
    Output: []

    Now extract the entities from the following user input:
    """ + user_input

    # Initialize the structured model
    llm = ChatOpenAI(model="gpt-4o-mini", max_completion_tokens=20)
    structured_llm = llm.with_structured_output(EntitiesToForget)

    try:
        response = structured_llm.invoke(instruction)
        extracted = response.entities
    except Exception:
        extracted = []

    # print("Entities to delete:", extracted)
    return {
        "personal_info_extracted": extracted
    }


In [12]:
extract_delete_entity({"messages": [HumanMessage(content= "I don't like to use Slack")]})

{'personal_info_extracted': ['Communication Preference: Slack']}

In [13]:
def forget_logic(state: AgentState) -> AgentState:
    entities = [item for item in state['personal_info_extracted']]
    namespace = ("user", USER_ID)
    key = "semantic_memory"
    print(f"Deleting these entities: {entities}")
    store.delete(namespace, key, entities)

In [ ]:
class Entities(BaseModel):
    entities: List[str] = Field(..., description="List of entities the user wants to add to memory")

def personal_info_extractor(state: AgentState) -> AgentState:
    """
    Extracts personal info from the user's message using few-shot examples.
    """
    message = state["messages"][-1].content

    # A few-shot style system prompt for extraction:
    extractor_prompt = """
    You are an intelligent extractor that identifies and summarizes personal user information for long-term memory storage. Your goal is to help the LLM provide more personalized responses in the future.

    Instructions:
    - Read the user's input carefully.
    - Extract any relevant personal details, such as:
    - Name, location, profession, and age
    - Interests, hobbies, goals, future plans
    - Food preferences (likes/dislikes)
    - Emotional states, values, or beliefs
    - Mention of past experiences (e.g., education, travel)
    - Any unique or identifying details

    Output Format:
    - Return a concise, comma-separated list of attributes in the format: 
    "Attribute: Value"
    - If multiple values exist, separate them with commas.
    - If no personal info is found, return: "No personal info found."

    Examples:

    User: "I am John and I live in Seattle."
    Output: ["Name: John", "Location: Seattle"]

    User: "Hey, I'm Lucy. I love eating bananas!"
    Output: ["Name: Lucy", "Food Preference: bananas"]

    User: "I am planning to apply to grad school in Germany next year."
    Output: ["Goal: apply to grad school, Germany"]

    User: "Just a random statement about the weather."
    Output: []

    Now extract personal information from the following user input:
    """ + message
   
    llm = ChatOpenAI(model="gpt-4o-mini", max_completion_tokens=20)
    structured_llm = llm.with_structured_output(Entities)

    try:
        response = structured_llm.invoke(extractor_prompt)
        # print(response)
        extracted = response.entities
    except Exception:
        extracted = []

    # print("Entities to Add:", extracted)


    state["personal_info_extracted"] = extracted
    # print(state["personal_info_extracted"])
    return state

In [15]:
personal_info_extractor({"messages": [HumanMessage(content="hi, I am Ashutosh. I love pizzas")]})

{'messages': [HumanMessage(content='hi, I am Ashutosh. I love pizzas', additional_kwargs={}, response_metadata={})],
 'personal_info_extracted': ['Name: Ashutosh', 'Food Preference: pizzas']}

In [16]:
class InfoNoveltyGrade(BaseModel):
    score: Literal["yes", "no"] = Field()

def personal_info_duplicate_classifier(state: AgentState) -> AgentState:
    """
    Checks if the newly extracted info is already in the store or not.
    If 'Yes', it's new info. If 'No', it's a duplicate.
    """
    new_info = state.get("personal_info_extracted", "")
    namespace = ("user", USER_ID)
    key = "semantic_memory"
    results = store.get(namespace, key)
    old_info_list = [doc for doc in results]

    system_msg = """You are a classifier that checks if the new personal info is already stored.
If the new info adds anything new, respond 'Yes'. Otherwise 'No'."""

    old_info_str = "\n".join(old_info_list) if old_info_list else "No stored info so far."
    human_template = """New info:\n{new_info}\n
Existing memory:\n{old_info}\n
Answer ONLY 'Yes' if the new info is unique. Otherwise 'No'."""
    human_msg = human_template.format(new_info=new_info, old_info=old_info_str)

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_msg),
            ("human", "{human_msg}"),
        ]
    )
    llm = ChatOpenAI(model="gpt-4o-mini", max_completion_tokens=50).with_structured_output(InfoNoveltyGrade)
    chain = prompt | llm
    result = chain.invoke({"human_msg": human_msg})
    state["new_info"] = result.score.strip()
    return state

In [17]:
def personal_info_deduper_router(state: AgentState) -> Literal["personal_info_storer", "retrieve_memories"]:
    """
    If 'Yes', store the new info. Otherwise skip storing.
    """
    if state["new_info"].lower() == "yes":
        return "personal_info_storer"
    return "retrieve_memories"

In [18]:
def personal_info_storer(state: AgentState) -> AgentState:
    """
    Stores the new personal info in memory if it exists.
    """
    extracted = state.get("personal_info_extracted")
    
    if extracted:
        namespace = ("user", USER_ID)
        store.put(namespace, key, {"text": extracted})
    return state

In [19]:
def retrieve_memories(state: AgentState) -> AgentState:
    """
    Retrieves all personal info from the store and aggregates into 'collected_memories'.
    """
    results = store.get(("user", USER_ID), key)
    memory_strs = [doc for doc in results]
    state["collected_memories"] = "\n".join(memory_strs)
    return state

In [20]:
def log_personal_memory(state: AgentState) -> AgentState:
    """
    Logs the memory to stdout for debugging (optional).
    """
    print("----- Logging Personal Memory -----")
    if state["collected_memories"]:
        for i, line in enumerate(state["collected_memories"].split("\n"), start=1):
            print(f"[Memory {i}] {line}")
    else:
        print("[Memory] No personal info stored yet.")
    return state

In [21]:
def call_model(state: AgentState) -> AgentState:
    """
    Final LLM call that uses the collected memories in a SystemMessage.
    """
    personal_info = state.get("collected_memories", "")
    system_msg = SystemMessage(
        content=f"You are a helpful assistant. The user has shared these personal details:\n{personal_info}"
    )
    all_messages = [system_msg] + list(state["messages"])
    llm = ChatOpenAI(model="gpt-4o-mini", max_completion_tokens=50)
    response = llm.invoke(all_messages)
    state["messages"] = state["messages"] + [response]
    return state

In [22]:
workflow = StateGraph(AgentState)
workflow.add_node("personal_info_classifier", personal_info_classifier)
workflow.add_node("classify_add_or_delete", classify_add_or_delete)
workflow.add_node("extract_delete_entity", extract_delete_entity)
workflow.add_node("forget_logic", forget_logic)
workflow.add_node("personal_info_extractor", personal_info_extractor)
workflow.add_node("personal_info_duplicate_classifier", personal_info_duplicate_classifier)
workflow.add_node("personal_info_storer", personal_info_storer)
workflow.add_node("retrieve_memories", retrieve_memories)
workflow.add_node("log_personal_memory", log_personal_memory)
workflow.add_node("call_model", call_model)

workflow.add_edge(START, "personal_info_classifier")
workflow.add_conditional_edges(
    "personal_info_classifier",
    personal_info_router,
    {
        "classify_add_or_delete": "classify_add_or_delete",
        "retrieve_memories": "retrieve_memories",
    },
)

workflow.add_conditional_edges(
    "classify_add_or_delete",
    route_add_or_delete,
    {
        "extract_delete_entity": "extract_delete_entity",
        "personal_info_extractor": "personal_info_extractor"
    }
)
workflow.add_edge("personal_info_extractor", "personal_info_duplicate_classifier")
workflow.add_edge("extract_delete_entity","forget_logic")
workflow.add_edge("forget_logic", "retrieve_memories")
workflow.add_conditional_edges(
    "personal_info_duplicate_classifier",
    personal_info_deduper_router,
    {
        "personal_info_storer": "personal_info_storer",
        "retrieve_memories": "retrieve_memories",
    },
)

workflow.add_edge("personal_info_storer", "retrieve_memories")
workflow.add_edge("retrieve_memories", "log_personal_memory")
workflow.add_edge("log_personal_memory", "call_model")
workflow.add_edge("call_model", END)

# workflow.set_entry_point("personal_info_classifier")

checkpointer = MemorySaver()
# graph = workflow.compile(checkpointer=checkpointer, store=store)
graph = workflow.compile(checkpointer=checkpointer)

In [23]:
# graph

In [27]:
# input_data_1 = {"messages": [HumanMessage(content="Hi, I am Ashutosh. I am an AI Engineer")]}
input_data_1 = {"messages": [HumanMessage(content="I am no longer an AI Engineer")]}
response = graph.invoke(input=input_data_1, config={"configurable": {"thread_id": 1}})

Deleting these entities: ['Job Title: AI Engineer']
----- Logging Personal Memory -----
[Memory 1] Name: Ashutosh


In [28]:
response

{'messages': [HumanMessage(content='Hi, I am Ashutosh. I am an AI Engineer', additional_kwargs={}, response_metadata={}, id='0278f460-c38f-4c67-938f-113f0e93b646'),
  AIMessage(content="Hello, Ashutosh! It's great to meet you. As an AI Engineer, you must be working on some interesting projects. What specific areas of AI do you focus on?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 48, 'total_tokens': 84, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BzqvtSdUfOafljFKzBcaG6eSsUgep', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--ad861cc3-1dad-4497-8671-d3b6894cb5b5-0', usage_metadata={'input_tokens': 48, 'output_tokens': 36, 'total_tok

In [29]:
response["messages"][-1].content

'Got it, Ashutosh! If you’d like to share what you’re currently doing or what your new role is, I’d love to hear about it.'